In [1]:
import pandas as pd
from pandas.io import gbq

In [11]:
table_df = gbq.read_gbq("SELECT * FROM `umg-comm-tech-dev.fixed_playlists_data.playlists_list`",
                       project_id = "umg-comm-tech-dev", dialect = 'standard')
table_df.head()

,artists,genre,playlist_owner,release_date,release_title,repertoire_pools,status,streaming_services,take_down_date,territory,upc
0,None,None,None,2016-11-23 00:00:00+00:00,Christmas,None,None,None,None,None,00602557103601
1,None,None,None,2016-12-09 00:00:00+00:00,Christmas Blues,None,None,None,None,None,00602557342697
2,None,None,None,2016-12-09 00:00:00+00:00,Christmas Jazz,None,None,None,None,None,00602557342703
3,None,None,None,2017-01-27 00:00:00+00:00,Crowd Pleasers,None,None,None,None,None,00602557414387
4,None,None,None,2017-02-03 00:00:00+00:00,Let Me Love You Old School,None,None,None,None,None,00602557437638


In [12]:
table_df.describe(include='all')

,artists,genre,playlist_owner,release_date,release_title,repertoire_pools,status,streaming_services,take_down_date,territory,upc
count,514,514,514,692,692,514,514,454,58,454,692
unique,5,33,35,132,616,15,7,3,26,11,692
top,Various artists,Jazz,William Freeman,2019-04-19 00:00:00+00:00,Bossa Nova Hits,Catalogue,DELIVERED,Spotify;Deezer;Napster;7digital;Amazon;Tidal;Y...,2018-10-09 00:00:00+00:00,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;EC;ES;FI;F...,00600406840011
freq,509,175,122,36,5,253,467,427,13,269,1


In [13]:
dtype_dict = {
    'upc':str
}

col_names = ['upc','release_title','artists', 'genre', 'status', 'release_date', 'take_down_date', 'playlist_owner', 
       'repertoire_pools', 'streaming_services', 'territory']

In [14]:
new_data = pd.read_csv('/Users/meshchd/Downloads/compilations-2019-05-28.csv', index_col=None, dtype=dtype_dict,
                      header=0, names=col_names)
new_data.head()

,upc,release_title,artists,genre,status,release_date,take_down_date,playlist_owner,repertoire_pools,streaming_services,territory
0,NaN,Jazz Standards: The 1920s,Various artists,Jazz,DRAFT,2018-05-11,NaN,Kayla Schmandt,Catalogue,Spotify;Deezer;Napster;7digital;Amazon,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;DO;EC;ES;F...
1,00600406836304,Blues For Breakfast,Various artists,Blues,DELIVERED,2018-05-04,NaN,James Burnett,Catalogue,Spotify;Deezer;Napster;7digital;Amazon,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;DO;EC;ES;F...
2,NaN,Punk Hits,Various artists,Punk,CLOSED,2018-05-07,NaN,Kayla Schmandt,Catalogue,Spotify;Deezer;Napster;7digital;Amazon,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;DO;EC;ES;F...
3,00600406836311,Midnight Jazz,Various artists,Jazz,DELIVERED,2018-05-11,NaN,Kayla Schmandt,Classical / Jazz,Spotify;Deezer;Napster;7digital;Amazon,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;DO;EC;ES;F...
4,00600406836366,Smooth Jazz,Various artists,Jazz,DELIVERED,2018-06-08,2018-10-10,Kayla Schmandt,Classical / Jazz,Spotify;Deezer;Napster;7digital;Amazon,AR;AT;AU;BO;BR;CA;CH;CL;CO;CR;DE;DK;DO;EC;ES;F...


In [15]:
truly_new = new_data[(new_data.upc.isnull()==False)&(new_data.upc.isin(table_df.upc.unique())==False)]

truly_new['release_date'] = pd.to_datetime(truly_new.release_date)
truly_new['take_down_date'] = pd.to_datetime(truly_new.take_down_date)

In [16]:
all_data = pd.concat([table_df,truly_new], ignore_index=True, sort=True)
all_data['release_date'] = pd.to_datetime(all_data.release_date)
all_data['take_down_date'] = pd.to_datetime(all_data.take_down_date)
all_data.upc.describe()

count                692
unique               692
top       00600406840011
freq                   1
Name: upc, dtype: object

In [17]:
all_data.to_gbq(destination_table="fixed_playlists_data.playlists_list",
                       project_id = "umg-comm-tech-dev", if_exists = 'replace')

1it [00:06,  6.01s/it]


In [18]:
len(truly_new.upc.unique())

0

In [19]:
all_data.shape

(692, 11)